In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
import os
import sys
import json
import logging

In [ ]:
sys.path.append(os.getcwd() + '/..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine

In [ ]:
log = logging.getLogger('papermill')
logging.basicConfig(level='INFO', format="%(message)s")

## Files and storage paths

In [ ]:
# Region of interest
reg = 1

# go down from rgi7_scripts/workflow
data_dir = '../../../rgi7_data/'

# Input dirctory
input_dir = os.path.join(data_dir, 'l3_rgi7a_tar')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l4_rgi7b0'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l5_rgi7b0_tar'))

### Load the input data

In [ ]:
# Read L2 files from GLIMS
shp = open_tar_shapefile(input_dir + f'/RGI{reg:02d}.tar.gz')

In [ ]:
orig_attrs = pd.DataFrame(shp.drop('geometry', axis=1))
orig_attrs.T;

In [ ]:
if 'conn_lvl' not in shp:
    shp['conn_lvl'] = 0

In [ ]:
odf = shp.copy()

In [ ]:
rename = {'area':'area_km2', 'CenLon': 'cenlon', 'CenLat': 'cenlat', 'glac_id':'glims_id'}

In [ ]:
odf = odf.rename(rename, axis=1)

In [ ]:
odf['area_km2'] = odf['area_km2'] * 1e-6
odf['glac_name'] = odf['glac_name'].where(odf['glac_name'] != 'None', '')

## Metadata 

In [ ]:
with open('rgi7_attributes_metadata.json', 'r') as infile:
    meta = json.load(infile)

In [ ]:
odf_new = odf[[]].copy().reset_index(drop=True)

for col, attrs in meta.items():
    if col not in odf:
        if attrs['datatype'] == 'str':
            odf_new[col] = ''
        elif attrs['datatype'] == 'float':
            odf_new[col] = np.NaN
        elif attrs['datatype'] == 'int':
            odf_new[col] = -999 
    else: 
        if attrs['datatype']:
            odf_new[col] = odf[col].astype(attrs['datatype'])
        else: 
            odf_new[col] = odf[col]

In [ ]:
odf_new = gpd.GeoDataFrame(odf_new)
odf_new.crs = odf.crs
odf_new = odf_new.reset_index(drop=True)
odf_new.T;

## Regions, subregions and RGI IDs

In [ ]:
odf_new['o1region'] = f'{reg:02d}'

In [ ]:
sreg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O2Regions')
sreg = gpd.read_file(sreg_file)
sreg = sreg.loc[sreg.o1region == f'{reg:02d}']
sreg

In [ ]:
baseid = f'RGI2000-v7.0-G-{reg:02d}-'
count = 0

rp = odf_new.representative_point()
rp = rp.to_frame('geometry')
rp['orig_index'] = odf_new.index

total = 0

for i, s in sreg.iterrows():
    
    intersect = gpd.overlay(rp, sreg.loc[[i]], how='intersection')
    odf_sreg = odf_new.loc[intersect['orig_index']].copy()
    odf_sreg['o2region'] = s['o2region']
    
    total += len(odf_sreg)
    
    if len(odf_sreg) == 0:
        # 19-05 Ross Ice Shelf one of them
        continue
    
    # Ids generation
    # Left most point and we start from there
    slon, slat = odf_sreg.loc[odf_sreg.cenlon == odf_sreg.cenlon.min()][['cenlon', 'cenlat']].iloc[0].values
    
    todo = odf_sreg.index.values
    todo_lon = odf_sreg['cenlon'].values
    todo_lat = odf_sreg['cenlat'].values
    ids = []
    while len(todo) > 0:
        dis = haversine(slon, slat, todo_lon, todo_lat)
        idm = np.argmin(dis)
        ids.append(todo[idm])
        slon, slat = todo_lon[idm], todo_lat[idm]
        todo = np.delete(todo, idm)
        todo_lon = np.delete(todo_lon, idm)
        todo_lat = np.delete(todo_lat, idm)
    
    assert len(ids) == len(odf_sreg)
    odf_sreg = odf_sreg.loc[ids].copy()
    
    odf_sreg['rgi_id'] = [baseid + f'{l+count:05d}' for l in range(len(odf_sreg))]
    count += len(odf_sreg)
    odf_new.loc[odf_sreg.index, 'rgi_id'] = odf_sreg['rgi_id']
    odf_new.loc[odf_sreg.index, 'o2region'] = odf_sreg['o2region']
    
odf_new = odf_new.sort_values(by='rgi_id').reset_index(drop=True)

assert odf_new['o2region'].isnull().sum() == 0
assert int(odf_new.iloc[-1]['rgi_id'].split('-')[-1]) == odf_new.iloc[-1].name
if len(odf_new['o2region'].unique()) != len(sreg):
    log.warning(f'RGI{reg:02d}: some subregions have NO glaciers in them')
else:
    log.info(f'RGI{reg:02d}: all subregions have glaciers in them')
assert len(odf_new) == len(odf)

In [ ]:
odf_new.iloc[:5].T;

## Write out and tar 

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
reg_file = reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0]

In [ ]:
dd = mkdir(f'{output_dir}/RGI2000-v7.0-G-{reg_file.long_code}/', reset=True)

print('Writing...')
odf_new.to_file(dd + f'RGI2000-v7.0-G-{reg_file.long_code}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI2000-v7.0-G-{reg_file.long_code}.tar.gz', '-C', output_dir, f'RGI2000-v7.0-G-{reg_file.long_code}']))

In [ ]:
odf_new;